In [1]:
import torchvision.models as models
import torch.nn as nn
import torch
import os
import cv2
from PIL import Image 
import glob
import torchvision
import torchvision.transforms as transforms
import datetime
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
from torchvision.utils import make_grid, save_image



from Grad_CAM.img_norm import Normalize, transform_img
from Grad_CAM.model_value import mv

In [2]:
use_pretrained=True # 학습 된 파라미터 사용
net = models.vgg16(pretrained=use_pretrained).cuda()

# 모델 네트워크 구성 출력
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
for i in net._modules.items():
    print(i[0])
    

features
avgpool
classifier


In [4]:
net._modules.keys()

odict_keys(['features', 'avgpool', 'classifier'])

In [5]:
def norm_preprocess(img_pth,means,stds): # img_pth 주면 img -> torch tensor type (1,3,224,224) 
    # means = [r,g,b] / std = [r,g,b]
    p_img = Image.open(img_pth) 
    tensor_img = torch.from_numpy(np.asarray(p_img)).permute(2,0,1).unsqueeze(0).float().div(255).cuda()
    # vgg -> 1,3,224,224
    # alex -> 1,3,227,227
    if tensor_img != (1,3,224,224) :
        tensor_img = F.interpolate(tensor_img,size=(224,224),mode='bilinear', align_corners=False)

    norm_mean_std = Normalize(mean=means, std=stds)
    nt_img = norm_mean_std(tensor_img)

    return nt_img , tensor_img

In [6]:
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [7]:
label_list = [] 
f = open('./imagenet_classes.txt','r')
while True:
    line = f.readline()
    if not line: break
    label_list.append(line.split('\n')[0])
f.close()

len(label_list)


1000

In [8]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
img_pth = glob.glob('./car_data/test_data/*.jpg')
model_pth = glob.glob('./vgg_pt/*pt')
save_pth = './car_data/grad_cam_result/vgg_pt/sportcar/'
label_idx = 817


In [9]:
for i in img_pth:
    name = i.split('/')[3]
    img,origin_img = norm_preprocess(i,means=[0.485, 0.456, 0.406], stds=[0.229, 0.224, 0.225])
    grad_cam = mv('features',net,img,origin_img,label_idx,name,save_pth,label_list)
    best_class,best_clas_name = grad_cam.make_img()
    if label_idx != best_class: 
        save_path = './car_data/grad_cam_result/vgg_pt/False_result'
        print('-------'*3)
        print(i)
        print(best_class)
        print(best_clas_name)
        grads_cam = mv('features',net,img,origin_img,best_class,name,save_path,label_list,re=True)
        a,b = grads_cam.make_img()


/compuworks/anaconda3/envs/torch18/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


---------------------
./car_data/test_data/000152.jpg
656
minivan
---------------------
./car_data/test_data/001273.jpg
436
beach wagon
---------------------
./car_data/test_data/000101.jpg
656
minivan
---------------------
./car_data/test_data/000851.jpg
882
vacuum
---------------------
./car_data/test_data/002211.jpg
511
convertible
---------------------
./car_data/test_data/000153.jpg
436
beach wagon
---------------------
./car_data/test_data/001221.jpg
436
beach wagon
---------------------
./car_data/test_data/001883.jpg
656
minivan
---------------------
./car_data/test_data/001688.jpg
882
vacuum
---------------------
./car_data/test_data/000141.jpg
656
minivan
---------------------
./car_data/test_data/000143.jpg
436
beach wagon
---------------------
./car_data/test_data/001958.jpg
511
convertible
---------------------
./car_data/test_data/000154.jpg
436
beach wagon
---------------------
./car_data/test_data/000194.jpg
656
minivan
---------------------
./car_data/test_data/000099.